In [18]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [19]:
  # Read files
  test_df = pd.read_csv('/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/True_24.csv')
  preds_df = pd.read_csv('/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/True_24.csv')
  admission_df = pd.read_csv('/content/drive/MyDrive/UM2ii/CXR Granular Bias/admissions.csv.gz', compression='gzip')
  admission_df = admission_df[['subject_id', 'race']]
  admission_df = admission_df.rename(columns={"race": "Race"})
  admission_df = admission_df.drop_duplicates()
  df = pd.merge(test_df, preds_df)
  df['subject_id'] = [i[2][1:] for i in df.path.str.split('/')]
  df['subject_id'] = df['subject_id'].astype(int)

  # There are individuals who inconsitently report their race (granular and course); will exclude
  admission_df = admission_df[~admission_df['subject_id'].duplicated(keep=False)]

  df = pd.merge(df, admission_df, how = "left", on = 'subject_id')
  df = df[df.Race != 'OTHER']
  df = df[df.Race != 'UNKNOWN']
  df = df[df.Race != 'UNABLE TO OBTAIN']
  df = df[df.Race != 'MULTIPLE RACE/ETHNICITY']
  df = df[df.Race != 'PATIENT DECLINED TO ANSWER']
  df = df[df.Race != 'AMERICAN INDIAN/ALASKA NATIVE']
  df = df[df.Race != 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER']

print("In our test set, we have " + str(sum(df.Race.value_counts())) + " individuals with a documented race/ethnicity." )
print("The proportion of each individuals in each documented race/ethnicity is as follows:")
print(df.Race.value_counts()/sum(df.Race.value_counts()))
print()
print("The number of each individuals in each documented race/ethnicity is as follows:")
print(df.Race.value_counts())

In our test set, we have 25888 individuals with a documented race/ethnicity.
The proportion of each individuals in each documented race/ethnicity is as follows:
WHITE                                 0.690938
BLACK/AFRICAN AMERICAN                0.172126
HISPANIC/LATINO - DOMINICAN           0.017228
WHITE - OTHER EUROPEAN                0.016533
HISPANIC/LATINO - PUERTO RICAN        0.015104
ASIAN - CHINESE                       0.012400
WHITE - RUSSIAN                       0.012361
BLACK/CAPE VERDEAN                    0.011627
BLACK/CARIBBEAN ISLAND                0.009155
ASIAN                                 0.009000
ASIAN - SOUTH EAST ASIAN              0.005060
BLACK/AFRICAN                         0.004674
HISPANIC OR LATINO                    0.003824
HISPANIC/LATINO - GUATEMALAN          0.002974
PORTUGUESE                            0.002472
ASIAN - ASIAN INDIAN                  0.002434
WHITE - BRAZILIAN                     0.002047
HISPANIC/LATINO - CUBAN               0.

In [20]:
print("The number of 'No Finding' Labels in each documented race/ethnicity is as follows:")
df = df[df.Race.notna()]
for race in sorted(df.Race.unique()):
  temp_df = df[df.Race==race]
  print(str(race) + ' ' + str(sum(temp_df['No Finding']==1)))

The number of 'No Finding' Labels in each documented race/ethnicity is as follows:
ASIAN 93
ASIAN - ASIAN INDIAN 30
ASIAN - CHINESE 122
ASIAN - KOREAN 18
ASIAN - SOUTH EAST ASIAN 56
BLACK/AFRICAN 69
BLACK/AFRICAN AMERICAN 2089
BLACK/CAPE VERDEAN 151
BLACK/CARIBBEAN ISLAND 71
HISPANIC OR LATINO 59
HISPANIC/LATINO - CENTRAL AMERICAN 4
HISPANIC/LATINO - COLUMBIAN 11
HISPANIC/LATINO - CUBAN 19
HISPANIC/LATINO - DOMINICAN 158
HISPANIC/LATINO - GUATEMALAN 30
HISPANIC/LATINO - HONDURAN 12
HISPANIC/LATINO - MEXICAN 16
HISPANIC/LATINO - PUERTO RICAN 200
HISPANIC/LATINO - SALVADORAN 17
PORTUGUESE 24
SOUTH AMERICAN 13
WHITE 6304
WHITE - BRAZILIAN 14
WHITE - EASTERN EUROPEAN 20
WHITE - OTHER EUROPEAN 155
WHITE - RUSSIAN 123


In [21]:
from scipy import stats
def wauc(true, preds):
  test_df = pd.read_csv(true)
  preds_df = pd.read_csv(preds)
  return metrics.roc_auc_score(y_true = test_df.drop('path', axis=1), y_score = preds_df.drop('path', axis=1),  average='macro', multi_class='ovr')


auc_24 = wauc('/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/True_24.csv', '/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/preds_24.csv')
auc_36 = wauc('/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/True_36.csv', '/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/preds_36.csv')
auc_70 = wauc('/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/True_70.csv', '/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/preds_70.csv')
auc_88 = wauc('/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/True_88.csv', '/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/preds_88.csv')
auc_100 = wauc('/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/True_100.csv', '/content/drive/MyDrive/UM2ii/CXR Granular Bias/MIMIC/preds_100.csv')

print('Average AUC of MIMIC-trained models applied onto MIMIC test set: ' + str(np.mean([auc_24, auc_36, auc_70, auc_88, auc_100])))
confidence_interval = stats.t.interval(0.95, len([auc_24, auc_36, auc_70, auc_88, auc_100])-1, loc=np.mean([auc_24, auc_36, auc_70, auc_88, auc_100]), scale=stats.sem([auc_24, auc_36, auc_70, auc_88, auc_100]))

# Print the confidence interval
print("95% Confidence Interval:", confidence_interval)

Average AUC of MIMIC-trained models applied onto MIMIC test set: 0.8277203537531038
95% Confidence Interval: (0.8259063416143976, 0.8295343658918101)
